# Import

In [ ]:
import numpy as np
import pandas as pd
# import gdown
from scipy import spatial
import spacy
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words("english")
from tqdm.notebook import tqdm
from random import shuffle
# from rouge import Rouge
from tqdm.notebook import tqdm
import re
import os
import torch
import pandas as pd
# from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import copy
from tqdm.notebook import tqdm

In [ ]:
model_type = 't5-base
model_location = # path
config_path = # path
tokenizer_path = # path

In [ ]:
%%capture
from transformers import AutoTokenizer, AutoConfig, AutoModelForSeq2SeqLM, BartTokenizer
# config = AutoConfig.from_pretrained(config_path)
tokenizer = BartTokenizer.from_pretrained(tokenizer_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_location)
config = model.config   

In [ ]:
model.to('cpu')

In [ ]:
import json
f = open(PATH,encoding="utf8")
data = json.load(f)
f.close()

# Code

In [ ]:
# params
device = 'cpu'

In [ ]:
def process(sents):
    final_list = []
    for sent in sents:
        if(len(sent)<4): continue
        if sent.isnumeric(): continue
        # print(sent)
        sent = " ".join(sent.split())
        # print(sent)
        final_list.append(sent.replace('\r',''))
    return final_list


stopwords = ['what', 'who','overview', 'overviews', 'where','a', 'an']
def filter_words(sen):
    querywords = sen.split()
    resultwords  = [word for word in querywords if word.lower() not in stopwords]
    result = ' '.join(resultwords)
    result = result.replace('?',"")
    result = result.replace('-',"")
    return result

In [ ]:
def convert_to_nl(all_patterns):
    for i, data in enumerate(all_patterns):
        if 't5' in model_type:
            prefix = "summarize: "
        else:
            prefix = ""
        if(len(data['topic'])<3): continue
        input_text =  prefix + 'what is ' + data['topic'] + '; ' + data['text']
        encoding = tokenizer.encode_plus(input_text,pad_to_max_length=True, return_tensors="pt", max_length=512)
        input_ids, attention_masks = encoding["input_ids"].to('cuda'), encoding["attention_mask"].to('cuda')
        beam_outputs = model.generate(
        input_ids=input_ids, attention_mask=attention_masks,
        do_sample=True,
        max_length=120,
        top_k=10000,
        top_p=0.98,
        early_stopping=False)
        for beam_output in beam_outputs:
            all_patterns[i]['definition'] = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    return all_patterns

In [ ]:
def what_is_module(sent):
    pattern = re.compile(r'what.is[^a-z]', re.I)
    corpus = []
    for ind, sen in enumerate(sent):
        sen = " ".join(sen.split())
        if pattern.match(sen):
            if(len(sen.split())>6):continue
            matches = pattern.match(sen)
            x = dict()
            x['text'] = ''
            x['sentence'] = sen
            if(len(sen)<9):continue
            if(sen[matches.span()[1]:][-1] =='?' or sen[matches.span()[1]:][-1] =='.'):
                x['topic'] =  filter_words(sen[matches.span()[1]:][:-1])
            else: x['topic'] =  filter_words(sen[matches.span()[1]:])
            for i in range(include_lines):
                if ((ind + i) < len(sent)-1): 
                    x['text'] += sent[i+ind+1]
                    x['text'] += '. '
            corpus.append(x)
    return corpus

In [ ]:
def overview_module(sent):
    corpus = []
    for ind, sen in enumerate(sent):
        if "overview" in sen:
            if(len(sen.split(" "))>5 or 'http' in sen):continue
            x = dict()
            x['topic'] = filter_words(sen)
            x['sentence'] = sen
            x['text'] = ''
            for i in range(include_lines):
                if ((ind + i) < len(sent)-1): 
                    x['text'] += sent[i+ind+1]
                    x['text'] += '. '
            corpus.append(x)
    return corpus

In [ ]:
def extract_patterns(text, include_lines =6):
    text = text.replace(r'\r','')
    sent = process(text.split('\n'))
    include_2 = 1
    all_patterns = []
    all_patterns.extend(what_is_module(sent))
    all_patterns.extend(overview_module(sent))

    all_patterns = convert_to_nl(all_patterns)
    return all_patterns

In [ ]:
include_lines = 6
pattern_exist = []
for ind, ppt in enumerate(tqdm(data)):
    x = dict()
    content = extract_patterns(ppt['FileContent'])
    if(len(content)>0):
        x["data"] = content
        x['id'] = ppt['Id']
        x['url'] = ppt['Url']
        pattern_exist.append(x)

In [ ]:
pattern_exist[0]

In [ ]:
def_list = []
for ppt in pattern_exist:
    for topics in ppt['data']:
        try:def_list.append([ppt['id'],ppt['url'],topics['topic'], topics['text'], topics['definition']])
        except: continue

In [ ]:
dz = pd.DataFrame(def_list, columns = ['id','url', 'topic','text','definition'])

In [ ]:
dz

In [ ]:
dz = pd.DataFrame(def_list, columns = ['id','url', 'topic','text','definition'])
dz.to_csv(PATH, index = False)

In [ ]:
cnt =0 
for i in pattern_exist:
    for j in i['data']:
        print(j['topic'])
        cnt+=1
cnt

# Extra Exp

In [ ]:
#get NER

from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

def get_continuous_chunks(text):
    chunked = ne_chunk(pos_tag(word_tokenize(text)))
    continuous_chunk = []
    current_chunk = []
    for i in chunked:
            if type(i) == Tree:
                    current_chunk.append(" ".join([token for token, pos in i.leaves()]))
            if current_chunk:
                    named_entity = " ".join(current_chunk)
                    if named_entity not in continuous_chunk:
                            continuous_chunk.append(named_entity)
                            current_chunk = []
            else:
                    continue
    return continuous_chunk
